In [5]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# La barra azul representa el porcentaje de asignaturas que SÍ pueden ser acomodadas en esa sala (aforo suficiente)
# La barra gris representa el porcentaje de asignaturas que NO pueden ser acomodadas (aforo insuficiente)

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def get_all_courses(profesores_input):
    """Extrae todas las asignaturas únicas con sus vacantes."""
    courses = []
    for profesor in profesores_input:
        for asignatura in profesor['Asignaturas']:
            courses.append({
                'codigo': asignatura['CodigoAsignatura'],
                'nombre': asignatura['Nombre'],
                'vacantes': asignatura['Vacantes']
            })
    return courses

def analyze_room_capacity(salas_input, courses):
    """Analiza qué asignaturas podrían ser alojadas óptimamente en cada sala."""
    try:
        room_stats = {}
        
        # Para cada sala
        for sala in salas_input:
            codigo = sala['Codigo']
            capacidad = sala['Capacidad']
            aforo_suficiente = 0
            aforo_insuficiente = 0
                
            # Comparar con cada asignatura
            for course in courses:
                if capacidad >= course['vacantes']:
                       aforo_suficiente += 1
                else:
                    aforo_insuficiente += 1
                
            room_stats[codigo] = {
                'capacidad': capacidad,
                'aforo_suficiente': aforo_suficiente,
                'aforo_insuficiente': aforo_insuficiente,
                'total': aforo_suficiente + aforo_insuficiente
            }
        
        return room_stats
    
    except Exception as e:
        print(f"Error al analizar la capacidad de salas: {str(e)}")
        return None

def create_horizontal_bar_chart(room_stats):
    """Crea un gráfico de barras horizontales compuesto con mayor grosor y separación."""

    # Convertir a DataFrame
    df = pd.DataFrame.from_dict(room_stats, orient='index')

    # Calcular porcentajes
    df['pct_aforo_suficiente'] = (df['aforo_suficiente'] / df['total'] * 100).round(2)
    df['pct_aforo_insuficiente'] = (df['aforo_insuficiente'] / df['total'] * 100).round(2)

    # Ordenar por porcentaje de asignaturas óptimas de mayor a menor
    df_sorted = df.sort_values('pct_aforo_suficiente', ascending=True)

    # Crear gráfico
    fig, ax = plt.subplots(figsize=(15, 30))

    # Posición de las barras con espaciado
    y_pos = np.arange(len(df_sorted)) * 1.5  # Espaciado entre filas

    # Crear barras con mayor grosor
    bar_height = 1  # Ajuste del grosor de las barras
    bars1 = ax.barh(y_pos, df_sorted['pct_aforo_suficiente'], label='Aforo Suficiente', color='#00629B', height=bar_height)
    bars2 = ax.barh(y_pos, df_sorted['pct_aforo_insuficiente'], left=df_sorted['pct_aforo_suficiente'],
                    label='Aforo Insuficiente', color='#A9A9A9', height=bar_height)

    # Configurar aspecto
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f"{idx} ({row['capacidad']})" for idx, row in df_sorted.iterrows()])

    # Eliminar bordes superior y derecho
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Añadir etiquetas en las barras
    for i, (opt, no_opt) in enumerate(zip(df_sorted['pct_aforo_suficiente'], df_sorted['pct_aforo_insuficiente'])):
        # Etiqueta para óptimas
        if opt > 0:
            ax.text(opt / 2, y_pos[i], f"{opt}%", ha='center', va='center', color='white', fontsize=10)

        # Etiqueta para no óptimas
        if no_opt > 0:
            ax.text(opt + no_opt / 2, y_pos[i], f"{no_opt}%", ha='center', va='center', color='black', fontsize=10)

    # Títulos y etiquetas
    plt.title('Aforo Potencial de las Salas para Asignaturas', fontsize=14)
    plt.xlabel('Porcentaje de Asignaturas', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)

    # Ajustar diseño y guardar
    plt.tight_layout()
    plt.savefig('Aforo_salas.png', bbox_inches='tight')
    plt.close()

    return True

def create_summary_table(room_stats):
    """Crea y muestra una tabla resumen de las estadísticas."""
    try:
        # Crear DataFrame
        df = pd.DataFrame.from_dict(room_stats, orient='index')
        
        # Calcular porcentajes
        df['pct_aforo_suficiente'] = (df['aforo_suficiente'] / df['total'] * 100).round(2)
        df['pct_aforo_insuficiente'] = (df['aforo_insuficiente'] / df['total'] * 100).round(2)
        
        # Ordenar por cantidad de asignaturas óptimas
        df_sorted = df.sort_values('aforo_suficiente', ascending=False)
        
        # Mostrar tabla
        print("\nResumen de Capacidad de Salas:")
        print("\nSala | Capacidad | aforo_suficiente (%) | No aforo_suficiente (%)")
        print("-" * 50)
        
        for idx, row in df_sorted.iterrows():
            print(f"{idx:6} | {row['capacidad']:9} | {row['pct_aforo_suficiente']:10.2f} | {row['pct_aforo_insuficiente']:11.2f}")
        
        return df_sorted
        
    except Exception as e:
        print(f"Error al crear la tabla resumen: {str(e)}")
        return None

def create_capacity_groups_summary(df):
    """
    Crea un resumen agrupando las salas por rangos de capacidad.
    """
    # Definir los rangos de capacidad
    ranges = [
        (0, 20, 'Muy pequeña'),
        (21, 30, 'Pequeña'),
        (31, 40, 'Mediana-Baja'),
        (41, 50, 'Mediana'),
        (51, 60, 'Mediana-Alta'),
        (61, float('inf'), 'Grande')
    ]
    
    # Crear columna de categoría
    df['categoria'] = pd.cut(
        df['capacidad'], 
        bins=[r[0] for r in ranges] + [float('inf')],
        labels=[r[2] for r in ranges],
        include_lowest=True
    )
    
    # Crear resumen por grupo
    group_summary = df.groupby('categoria').agg({
        'Salas': 'count',
        'capacidad': ['min', 'max'],
        'pct_aforo_suficiente': ['mean', 'min', 'max'],
    }).round(2)
    
    # Renombrar columnas para mejor legibilidad
    group_summary.columns = [
        'Cantidad_Salas', 
        'Capacidad_Min', 'Capacidad_Max',
        'Pct_Aforo_Promedio', 'Pct_Aforo_Min', 'Pct_Aforo_Max',
    ]
    
    return group_summary

def save_results(room_stats, df_summary):
    """Guarda los resultados en archivos CSV y JSON."""
    try:
        # Renombrar la columna de índice antes de guardar
        df_summary.index.name = 'Salas'
        
        # Guardar CSV
        df_summary.to_csv('resumen_aforo_salas.csv')
        
        # Crear y guardar el resumen por grupos
        group_summary = create_capacity_groups_summary(df_summary.reset_index())
        group_summary.to_csv('resumen_grupos_salas.csv')
        
        # Guardar JSON
        with open('metricas_aforo_salas.json', 'w', encoding='utf-8') as f:
            json.dump(room_stats, f, ensure_ascii=False, indent=2)
        
        # Imprimir resumen
        print("\nResumen por Grupos de Capacidad:")
        print("\nInterpretación del análisis:")
        print("- Las salas se han agrupado por su capacidad para facilitar la planificación")
        print("- El porcentaje de aforo suficiente indica qué proporción de asignaturas pueden ser asignadas")
        print("- Un alto porcentaje indica que la sala es más versátil para la asignación")
        print("\nResultados por grupo:")
        print(group_summary.to_string())
        
        return True
        
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    profesores_input = load_json_file('../../agent_input/InputOfProfesores.json')
    salas_input = load_json_file('../../agent_input/InputOfSala.json')
    
    if profesores_input is None or salas_input is None:
        return
    
    # 2. Obtener todas las asignaturas
    courses = get_all_courses(profesores_input)
    
    # 3. Analizar capacidad de salas
    room_stats = analyze_room_capacity(salas_input, courses)
    if room_stats is None:
        return
    
    # 4. Crear tabla resumen
    df_summary = create_summary_table(room_stats)
    if df_summary is None:
        return
    
    # 5. Crear visualización
    if not create_horizontal_bar_chart(room_stats):
        return
    
    # 6. Guardar resultados
    if save_results(room_stats, df_summary):
        print("\nAnálisis de capacidad de salas completado exitosamente")
    else:
        print("Error al completar el análisis de capacidad de salas")

if __name__ == "__main__":
    main()


Resumen de Capacidad de Salas:

Sala | Capacidad | aforo_suficiente (%) | No aforo_suficiente (%)
--------------------------------------------------
CRP43  |      71.0 |     100.00 |        0.00
CRP33  |      71.0 |     100.00 |        0.00
KAUS1  |      68.0 |     100.00 |        0.00
K1     |      82.0 |     100.00 |        0.00
IC4    |      77.0 |     100.00 |        0.00
V205   |      65.0 |     100.00 |        0.00
V207   |      60.0 |      98.42 |        1.58
CM3    |      50.0 |      93.86 |        6.14
CM5    |      45.0 |      91.88 |        8.12
V103   |      45.0 |      91.88 |        8.12
V102   |      45.0 |      91.88 |        8.12
E6     |      45.0 |      91.88 |        8.12
E1     |      45.0 |      91.88 |        8.12
CM4    |      45.0 |      91.88 |        8.12
LC-03  |      40.0 |      87.13 |       12.87
V105   |      40.0 |      87.13 |       12.87
LC6    |      40.0 |      87.13 |       12.87
LAMB   |      40.0 |      87.13 |       12.87
K3     |      40.0 |  

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20400\1621770496.py:173: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_summary = df.groupby('categoria').agg({
